In [192]:
import math

def generatePatternBitmasks(pattern, m):
    count = math.ceil(m / 64.0)
    len_ = 256 * count  # CHAR_MAX is 255 in C, plus 1 for the range

    # Initialize the pattern bitmasks
    patternBitmasks = [2**64 - 1] * len_

    # Update the pattern bitmasks
    for i in range(m):
        index = count - ((m - i - 1) // 64) - 1
        char_index = ord(pattern[i])
        patternBitmasks[char_index * count + index] &= ~(1 << ((m - i - 1) % 64))

    return patternBitmasks


def generatePatternBitmasksACGT(pattern, m):
    count = math.ceil(m / 64.0)
    len_ = 4 * count  # A, C, G, T

    # Initialize the pattern bitmasks
    patternBitmasks = [2**64 - 1] * len_

    # Update the pattern bitmasks
    for i in range(m):
        index = count - ((m - i - 1) // 64) - 1
        char_index = {"A": 0, "C": 1, "G": 2, "T": 3}.get(pattern[i].upper(), -1)
        if char_index != -1:
            patternBitmasks[char_index * count + index] &= ~(1 << ((m - i - 1) % 64))

    return patternBitmasks

def calculateScore(
    countM, countS, countOpen, countExtend, scoreM, scoreS, scoreOpen, scoreExtend
):
    return (
        (countM * scoreM)
        + (countS * scoreS)
        + (countOpen * (scoreOpen + scoreExtend))
        + (countExtend * scoreExtend)
    )

def genasmTB(n, k, count, tracebackMatrix, m, minError, mask, text):
    ed = 0
    CIGARstr = ""
    CIGARstr2 = ""
    MD = ""
    charCount = 0
    charCount2 = 0
    charCount3 = 0
    lastChar = "0"
    lastChar2 = "0"
    lastChar3 = "0"

    isFirst = True

    countM = 0
    countS = 0
    countD = 0
    countI = 0
    countOpen = 0
    countExtend = 0
    curPattern = m - 1
    curText = 0
    curError = minError

    while (curPattern >= 0) and (curError >= 0):
        ind = count - (curPattern // 64) - 1

        # affine-insertion
        if lastChar == "I" and (tracebackMatrix[curText][curError][2][ind] & mask) == 0:
            curPattern -= 1
            curError -= 1
            mask = 1 << (curPattern % 64)
            if lastChar == "I":
                charCount += 1
                countExtend += 1
            else:
                if not isFirst:
                    CIGARstr += str(charCount) + lastChar
                charCount = 1
                lastChar = "I"
                countOpen += 1
            if lastChar2 == "I":
                charCount2 += 1
            else:
                if not isFirst:
                    CIGARstr2 += str(charCount2) + lastChar2
                charCount2 = 1
                lastChar2 = "I"
            countI += 1
            ed += 1

        # affine-deletion
        elif (
            lastChar == "D" and (tracebackMatrix[curText][curError][3][ind] & mask) == 0
        ):
            curText += 1
            curError -= 1
            if lastChar == "D":
                charCount += 1
                countExtend += 1
            else:
                if not isFirst:
                    CIGARstr += str(charCount) + lastChar
                charCount = 1
                lastChar = "D"
                countOpen += 1
            if lastChar2 == "D":
                charCount2 += 1
            else:
                if not isFirst:
                    CIGARstr2 += str(charCount2) + lastChar2
                charCount2 = 1
                lastChar2 = "D"
            if lastChar3 == "M":
                MD += str(charCount3) + "^" + text[curText - 1]
                lastChar3 = "D"
                charCount3 = 0
            elif lastChar3 == "D":
                MD += text[curText - 1]
                lastChar3 = "D"
                charCount3 = 0
            else:
                MD += "^" + text[curText - 1]
                lastChar3 = "D"
                charCount3 = 0
            countD += 1
            ed += 1

        # match
        elif (tracebackMatrix[curText][curError][0][ind] & mask) == 0:
            curText += 1
            curPattern -= 1
            mask = 1 << (curPattern % 64)
            if lastChar == "M":
                charCount += 1
            else:
                if not isFirst:
                    CIGARstr += str(charCount) + lastChar
                charCount = 1
                lastChar = "M"
            if lastChar2 == "M":
                charCount2 += 1
            else:
                if not isFirst:
                    CIGARstr2 += str(charCount2) + lastChar2
                charCount2 = 1
                lastChar2 = "M"
            if lastChar3 == "M":
                charCount3 += 1
            else:
                charCount3 = 1
                lastChar3 = "M"
            countM += 1

        # substitution
        elif (tracebackMatrix[curText][curError][1][ind] & mask) == 0:
            curText += 1
            curPattern -= 1
            curError -= 1
            mask = 1 << (curPattern % 64)
            if lastChar == "S":
                charCount += 1
            else:
                if not isFirst:
                    CIGARstr += str(charCount) + lastChar
                charCount = 1
                lastChar = "S"
            if lastChar2 == "M":
                charCount2 += 1
            else:
                if not isFirst:
                    CIGARstr2 += str(charCount2) + lastChar2
                charCount2 = 1
                lastChar2 = "M"
            if lastChar3 == "M":
                MD += str(charCount3) + text[curText - 1]
                lastChar3 = "S"
                charCount3 = 0
            else:
                MD += text[curText - 1]
                lastChar3 = "S"
                charCount3 = 0
            countS += 1
            ed += 1

        # deletion
        elif (tracebackMatrix[curText][curError][3][ind] & mask) == 0:
            curText += 1
            curError -= 1
            if lastChar == "D":
                charCount += 1
                countExtend += 1
            else:
                if not isFirst:
                    CIGARstr += str(charCount) + lastChar
                charCount = 1
                lastChar = "D"
                countOpen += 1
            if lastChar2 == "D":
                charCount2 += 1
            else:
                if not isFirst:
                    CIGARstr2 += str(charCount2) + lastChar2
                charCount2 = 1
                lastChar2 = "D"
            if lastChar3 == "M":
                MD += str(charCount3) + "^" + text[curText - 1]
                lastChar3 = "D"
                charCount3 = 0
            elif lastChar3 == "D":
                MD += text[curText - 1]
                lastChar3 = "D"
                charCount3 = 0
            else:
                MD += "^" + text[curText - 1]
                lastChar3 = "D"
                charCount3 = 0
            countD += 1
            ed += 1

        # insertion
        elif (tracebackMatrix[curText][curError][2][ind] & mask) == 0:
            curPattern -= 1
            curError -= 1
            mask = 1 << (curPattern % 64)
            if lastChar == "I":
                charCount += 1
                countExtend += 1
            else:
                if not isFirst:
                    CIGARstr += str(charCount) + lastChar
                charCount = 1
                lastChar = "I"
                countOpen += 1
            if lastChar2 == "I":
                charCount2 += 1
            else:
                if not isFirst:
                    CIGARstr2 += str(charCount2) + lastChar2
                charCount2 = 1
                lastChar2 = "I"
            countI += 1
            ed += 1

        isFirst = False

    CIGARstr += str(charCount) + lastChar
    CIGARstr2 += str(charCount2) + lastChar2
    if lastChar3 == "M":
        MD += str(charCount3)

    return (
        ed,
        CIGARstr,
        CIGARstr2,
        MD,
        countM,
        countS,
        countD,
        countI,
        countOpen,
        countExtend,
    )


def genasmDC(text, pattern, k, scoreM, scoreS, scoreOpen, scoreExtend):
    m = len(pattern)
    n = len(text)

    max_64bit = 2**64 - 1

    patternBitmasks = generatePatternBitmasks(pattern, m)

    count = math.ceil(m / 64)
    rem = m % 64

    max1 = (1 << (rem - 1)) if rem > 0 else (1 << 63)

    # Initialize the bit arrays R
    R = [max_64bit] * ((k + 1) * count)

    tracebackMatrix = [
        [[[max_64bit for _ in range(count)] for _ in range(4)] for _ in range(k + 1)]
        for _ in range(n)
    ]

    oldR = R[:]

    # now traverse the text in opposite direction (i.e., forward), generate partial tracebacks at each checkpoint
    for i in range(n - 1, -1, -1):
        c = text[i]

        # copy the content of R to oldR
        oldR = R[:]

        curBitmask = patternBitmasks[ord(c) * count : ord(c) * count + count]

        # update R[0] by first shifting oldR[0] and then ORing with pattern bitmask
        R[0] = (oldR[0] << 1) & max_64bit
        for a in range(1, count):
            R[a - 1] |= oldR[a] >> 63
            R[a] = (oldR[a] << 1) & max_64bit

        for a in range(0, count):
            R[a] |= curBitmask[a]

        for a in range(count):
            tracebackMatrix[i][0][0][a] = R[a]
            tracebackMatrix[i][0][1][a] = max
            tracebackMatrix[i][0][2][a] = max
            tracebackMatrix[i][0][3][a] = max

        for d in range(1, k + 1):
            index = (d - 1) * count

            deletion = oldR[index : index + count]

            substitution = [(deletion[0] << 1) & max_64bit]
            for a in range(1, count):
                substitution.append(
                    (deletion[a - 1] >> 63) & max_64bit | (deletion[a] << 1) & max_64bit
                )

            insertion = [(R[index] << 1) & max_64bit]
            for a in range(1, count):
                insertion.append(
                    (R[index + a] >> 63) & max_64bit | (R[index + a] << 1) & max_64bit
                )

            index += count

            match = [(oldR[index] << 1) & max_64bit]
            for a in range(1, count):
                match.append(
                    (oldR[index + a] >> 63) & max_64bit
                    | (oldR[index + a] << 1) & max_64bit
                )

            match = [x | y for x, y in zip(match, curBitmask)]

            R[index : index + count] = [
                x & y & z & w
                for x, y, z, w in zip(deletion, substitution, insertion, match)
            ]

            for a in range(count):
                tracebackMatrix[i][d][0][a] = match[a]
                tracebackMatrix[i][d][1][a] = substitution[a]
                tracebackMatrix[i][d][2][a] = insertion[a]
                tracebackMatrix[i][d][3][a] = deletion[a]

    # print_matrix(tracebackMatrix, text, pattern)

    minError = -1
    mask = max1

    for t in range(k + 1):
        if (R[t * count] & mask) == 0:
            minError = t
            break

    if minError == -1:
        print("No alignment found!")
        return

    (
        ed,
        CIGARstr,
        CIGARstr2,
        MD,
        countM,
        countS,
        countD,
        countI,
        countOpen,
        countExtend,
    ) = genasmTB(n, k, count, tracebackMatrix, m, minError, mask, text)

    bitmacScore = (
        countM * scoreM
        - countS * scoreS
        - countOpen * (scoreOpen + scoreExtend)
        - countExtend * scoreExtend
    )

    print(f"ED:{ed}\tAS:{bitmacScore}\t{CIGARstr}\t{CIGARstr2}\t{MD}")


def genasm_aligner(text, pattern, k, scoreM, scoreS, scoreOpen, scoreExtend):
    genasmDC(text, pattern, k, scoreM, scoreS, scoreOpen, scoreExtend)

[18446744073709551479, 18446744073709551547, 18446744073709551581, 18446744073709551598]


In [194]:
genasm_aligner("GGTTGACTA", "TGTTACGG", 10, 0, 0, 0, 0)

# ED:3	AS:0	1M1D1M1S2M1I1M	1M1D4M1I1M	1^A1G3

ED:4	AS:0	1S3M1D2M2S	4M1D4M	G3^G2TA
